In [4]:
import sqlite3
import os

# Ruta absoluta o relativa a tu base de datos
# db_path = os.path.join('flask_trazos', 'instance', 'database.db')
db_path = r'C:\Users\francisco.contreras\Desktop\jupyter-projects\notebooks\APIs\flask_trazos_v3\flask_trazos\instance\avaluo.db'

# Conexión a la base de datos
conn = sqlite3.connect(db_path)

# Permitir acceder a las columnas como diccionario (opcional pero útil)
conn.row_factory = sqlite3.Row

# Crear cursor para ejecutar consultas
cursor = conn.cursor()


In [7]:
conn = sqlite3.connect(db_path)
conn.row_factory = sqlite3.Row  # permite acceder a los nombres de columna
cursor = conn.cursor()

In [1]:
import sqlite3

db_path = r'C:\Users\francisco.contreras\Desktop\jupyter-projects\notebooks\APIs\flask_trazos_v3\buzon\instance\avaluo.db'

# Abrimos la conexión
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Ejecutamos la consulta para obtener nombres de tablas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tablas = cursor.fetchall()

print("Tablas en la base de datos:")
for tabla in tablas:
    print(tabla[0])

# Cerramos la conexión cuando terminamos
conn.close()


Tablas en la base de datos:
avaluo_maestro
emision_avaluo
sqlite_sequence
emision_avaluo_v2


In [7]:
import sqlite3

conn = sqlite3.connect(r'C:\Users\francisco.contreras\Desktop\jupyter-projects\notebooks\APIs\flask_trazos_v3\buzon\instance\avaluo.db')
cursor = conn.cursor()

try:
    cursor.execute("ALTER TABLE emision_avaluo_v2 ADD COLUMN hash_archivo TEXT;")
    conn.commit()
    print("Columna hash_archivo agregada correctamente.")
except sqlite3.OperationalError as e:
    if "duplicate column name" in str(e).lower():
        print("La columna hash_archivo ya existe, no se agregó.")
    else:
        raise
finally:
    cursor.close()
    conn.close()


La columna hash_archivo ya existe, no se agregó.


In [8]:
import sqlite3

db_path = r'C:\Users\francisco.contreras\Desktop\jupyter-projects\notebooks\APIs\flask_trazos_v3\buzon\instance\avaluo.db'

conn = sqlite3.connect(db_path)
conn.row_factory = sqlite3.Row
cursor = conn.cursor()

cursor.execute("SELECT * FROM avaluo_maestro")
registros = cursor.fetchall()

for r in registros:
    print(dict(r))  # imprime cada registro como diccionario

conn.close()


{'id': 1, 'rfc': 'NNNN141414NNN', 'nombre': 'ELENA VARGAS', 'clave': 'AVAL-014', 'fecha': '2025-06-14', 'servidor': 'ROBERTO DÍAZ', 'fuente': 'INDAABIN', 'costo': 'PERITO CATORCE', 'fecha_pago': '2025-06-23', 'comprobante_numero': 'COMP-014', 'clc_numero': 'CLC-014', 'archivo_comprobante': 'comp_014.pdf', 'archivo_clc': 'clc_014.pdf', 'firma_digital': 'FIRMA_DUMMY_014', 'fecha_firmada': '2025-06-28'}
{'id': 2, 'rfc': 'OOOO151515OOO', 'nombre': 'FERNANDO MEJÍA', 'clave': 'AVAL-015', 'fecha': '2025-06-15', 'servidor': 'SANDRA MORALES', 'fuente': 'INDAABIN', 'costo': 'PERITO QUINCE', 'fecha_pago': '2025-06-24', 'comprobante_numero': 'COMP-015', 'clc_numero': 'CLC-015', 'archivo_comprobante': 'comp_015.pdf', 'archivo_clc': 'clc_015.pdf', 'firma_digital': 'FIRMA_DUMMY_015', 'fecha_firmada': '2025-06-29'}
{'id': 3, 'rfc': 'PPPP161616PPP', 'nombre': 'SUSANA PÉREZ', 'clave': 'AVAL-016', 'fecha': '2025-06-16', 'servidor': 'MIGUEL ANGEL', 'fuente': 'INDAABIN', 'costo': 'PERITO DIECISÉIS', 'fecha

In [8]:
!pip install paramiko

In [7]:
!pip install psycopg2

In [10]:
import paramiko

ssh_host = "172.27.23.81"
ssh_user = "admin"
ssh_password = "+*YMLMsabg#.042025"

# Script Python que se ejecutará en el servidor 81
remote_python_script = """
import psycopg2
try:
    conn = psycopg2.connect(
        host="172.25.30.1",
        port="5435",
        dbname="pviamaya",
        user="francisco_dev",
        password="Cambiame:)"
    )
    print("✅ Conexión exitosa a la base de datos.")
except psycopg2.Error as e:
    print("❌ Error al conectar a la base de datos:", e)
finally:
    if 'conn' in locals() and conn:
        conn.close()
        print("🔒 Conexión cerrada.")
"""

# Establece conexión SSH
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

try:
    print(f"🔐 Conectando por SSH a {ssh_host}...")
    client.connect(ssh_host, username=ssh_user, password=ssh_password)

    # Crear archivo temporal en el servidor remoto
    sftp = client.open_sftp()
    remote_path = "/tmp/test_pg_connection.py"
    with sftp.file(remote_path, 'w') as f:
        f.write(remote_python_script)
    sftp.close()

    # Ejecutar el script remoto
    print("🚀 Ejecutando script de conexión a PostgreSQL desde el servidor remoto...")
    stdin, stdout, stderr = client.exec_command(f"python3 {remote_path}")
    
    print("🖨️ Salida del script:")
    print(stdout.read().decode())
    print("⚠️ Errores (si hay):")
    print(stderr.read().decode())

finally:
    client.close()
    print("✅ Sesión SSH cerrada.")


🔐 Conectando por SSH a 172.27.23.81...
🚀 Ejecutando script de conexión a PostgreSQL desde el servidor remoto...
🖨️ Salida del script:
❌ Error al conectar a la base de datos: connection to server at "172.25.30.1", port 5435 failed: FATAL:  password authentication failed for user "francisco_dev"


⚠️ Errores (si hay):

✅ Sesión SSH cerrada.


In [11]:
import psycopg2

try:
    conn = psycopg2.connect(
        host="172.25.30.1",
        port="5435",
        dbname="pviamaya",
        user="francisco_dev",
        password="Cambiame:)"
    )
    conn.autocommit = True
    cursor = conn.cursor()

    # Crear el esquema si no existe
    cursor.execute("CREATE SCHEMA IF NOT EXISTS desarrollo;")
    print("✅ Esquema 'desarrollo' verificado o creado.")

    # Usar el esquema 'desarrollo'
    cursor.execute("SET search_path TO desarrollo;")
    print("📂 Cambiado al esquema 'desarrollo'.")

    # Crear tabla 'prueba'
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS prueba (
            id SERIAL PRIMARY KEY,
            nombre TEXT,
            creado_en TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
    """)
    print("✅ Tabla 'prueba' creada o ya existente en el esquema 'desarrollo'.")

    # 🟢 Conexión sigue abierta para seguir trabajando con `conn` y `cursor`
    print("🟢 Conexión abierta y lista para usar.")

except psycopg2.Error as e:
    print("❌ Error durante conexión o creación:", e)
    conn = None
    cursor = None


❌ Error durante conexión o creación: connection to server at "172.25.30.1", port 5435 failed: FATAL:  password authentication failed for user "francisco_dev"

